In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/src")

In [2]:
import pandas as pd
import numpy as np
import joblib
from scipy import stats

from config import exp_vaep_chain_output_path
from visualization.explore import *

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Load VAEP Data

In [3]:
exp_vaep_data = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/exp_vaep_values_v3.csv")
exp_vaep_data.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value
0,202101_BrisbaneLions_Sydney,1,2.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,NaN,0.459884,0.057641,0.000000,-0.000000,0.000000
1,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,NaN,0.000000,0.157755,-0.459884,-0.100114,-0.559998
2,202101_BrisbaneLions_Sydney,1,4.0,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,NaN,0.303480,0.128561,0.145725,-0.128561,0.017163
3,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,NaN,0.270377,0.084397,-0.033102,0.044165,0.011063
4,202101_BrisbaneLions_Sydney,1,6.0,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,NaN,0.233148,0.112946,-0.037229,-0.028549,-0.065778


In [4]:
exp_vaep_data[['exp_scores', 'exp_concedes']].sum()

exp_scores      823959.057114
exp_concedes    167376.456941
dtype: float64

In [5]:
exp_vaep_data[['exp_offensive_value', 'exp_defensive_value', 'exp_vaep_value']].sum()

exp_offensive_value    111349.191252
exp_defensive_value      1949.478212
exp_vaep_value         113298.669467
dtype: float64

In [6]:
exp_vaep_data.sort_values(by = 'exp_vaep_value', ascending=True).head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value
564388,202216_GoldCoast_Collingwood,88,634.0,2,521.0,2481.0,Collingwood,Brody Mihocek,73.0,7.0,-67.0,14.0,Kick,clanger,NaN,0.000000,0.032282,-5.215647,0.159560,-5.056087
82367,202106_GoldCoast_Sydney,139,1173.0,3,137.0,3815.0,Gold Coast,Josh Corbett,73.0,0.0,-66.0,13.0,Carry,effective,NaN,0.266753,0.023469,-4.520755,0.046820,-4.473935
575248,202217_GoldCoast_Richmond,251,1884.0,4,1223.0,6831.0,Richmond,Jason Castagna,58.0,2.0,-63.0,3.0,Kick,clanger,NaN,0.284475,0.116213,-4.343596,-0.032412,-4.376007
798099,202308_Geelong_Adelaide,62,432.0,1,1706.0,1706.0,Geelong,Brad Close,79.0,-6.0,-66.0,-11.0,Error,ineffective,NaN,0.000000,0.043277,-4.082661,0.014499,-4.068162
161732,202111_Sydney_Carlton,83,701.0,2,943.0,2906.0,Carlton,Harry McKay,72.0,4.0,-67.0,-4.0,Carry,effective,NaN,0.371210,0.060052,-4.065178,0.007518,-4.057661


In [7]:
exp_vaep_data['Season'] = exp_vaep_data['match_id'].apply(lambda x: int(x[:4]))
exp_vaep_2021 = exp_vaep_data[exp_vaep_data['Season'] == 2021]
exp_vaep_2022 = exp_vaep_data[exp_vaep_data['Season'] == 2022]
exp_vaep_2023 = exp_vaep_data[exp_vaep_data['Season'] == 2023]

In [8]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/scored_player_stats_v3.csv")

In [9]:
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats_2021 = player_stats[player_stats['Season'] == 2021]
player_stats_2022 = player_stats[player_stats['Season'] == 2022]
player_stats_2023 = player_stats[player_stats['Season'] == 2023]

Individual Actions

In [10]:
exp_vaep_data['exp_vaep_value'].max(), exp_vaep_data['exp_vaep_value'].min()

(6.5295453, -5.056087)

In [11]:
exp_vaep_data.sort_values('exp_vaep_value', ascending = False).head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value,Season
820308,202309_Sydney_Fremantle,97,760.0,2,967.0,2744.0,Fremantle,Josh Treacy,15.0,11.0,61.0,-1.0,Free For,effective,NaN,5.468151,0.314276,5.151494,1.378051,6.529545,2023
316762,202123_Adelaide_NorthMelbourne,194,1633.0,4,194.0,5540.0,North Melbourne,Nick Larkey,73.0,0.0,69.0,0.0,Free For,effective,NaN,5.683940,0.157392,4.058993,2.314244,6.373237,2021
807691,202308_Richmond_WestCoast,250,2011.0,4,1238.0,7141.0,West Coast,Bailey J. Williams,77.0,0.0,69.0,0.0,Contested Mark,effective,NaN,6.000000,0.078005,6.000000,-0.078005,5.921995,2023
475713,202209_StKilda_Geelong,106,918.0,2,1184.0,2975.0,St Kilda,Max King,72.0,4.0,75.0,0.0,Contested Mark,effective,NaN,6.000000,0.091021,6.000000,-0.091021,5.908979,2022
909708,202316_WestCoast_StKilda,104,900.0,2,1678.0,3668.0,St Kilda,Anthony Caminiti,80.0,0.0,75.0,0.0,Contested Mark,effective,NaN,6.000000,0.107851,6.000000,-0.107851,5.892149,2023


Match Level

In [12]:
def get_match_player_level_vaep(vaep_data, match_id, shots = True):
    
    match = get_match(vaep_data, match_id)
    if shots == False:
        match = match[match['action_type'] != "Shot"]
    match_vaep = get_player_vaep(match)
    match_vaep_ranking = get_match_rankings(match_vaep)
    
    return match_vaep_ranking

In [13]:
match_id = "2022F4_Geelong_Sydney"

In [14]:
match_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id)
match_vaep_ranking.sort_values('exp_vaep_value', ascending = False).head()

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_exp_vaep_value,normalised_exp_offensive_value,normalised_exp_defensive_value,z_exp_vaep_value,z_exp_offensive_value,z_exp_defensive_value
30,Patrick Dangerfield,Geelong,15.338971,15.861482,-0.522511,58,1.0,1.0,26.0,1.000000,1.000000,0.456320,3.349271,3.697657,-0.105565
5,Chad Warner,Sydney,10.876419,11.711944,-0.835525,69,2.0,2.0,31.0,0.775568,0.768003,0.419641,2.146988,2.481509,-0.284229
24,Mark Blicavs,Geelong,9.543365,8.093971,1.449394,49,3.0,4.0,6.0,0.708525,0.565726,0.687389,1.787841,1.421153,1.019969
13,Isaac Smith,Geelong,9.473450,11.122511,-1.649062,69,4.0,3.0,38.0,0.705009,0.735049,0.324311,1.769005,2.308758,-0.748583
32,Robbie Fox,Sydney,7.458894,3.341723,4.117172,59,5.0,18.0,1.0,0.603693,0.300032,1.000000,1.226251,0.028364,2.542696


In [15]:
match_noshot_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id, shots=False)
match_noshot_vaep_ranking.sort_values('exp_vaep_value', ascending = False).head()

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_exp_vaep_value,normalised_exp_offensive_value,normalised_exp_defensive_value,z_exp_vaep_value,z_exp_offensive_value,z_exp_defensive_value
30,Patrick Dangerfield,Geelong,15.668334,16.205397,-0.537063,57,1.0,1.0,29.0,1.000000,1.000000,0.454615,3.639994,4.182358,-0.186692
24,Mark Blicavs,Geelong,9.050938,7.502813,1.548125,47,2.0,4.0,6.0,0.672619,0.522626,0.698958,1.781390,1.441909,1.023631
5,Chad Warner,Sydney,8.842043,9.291757,-0.449714,66,3.0,3.0,27.0,0.662284,0.620757,0.464851,1.722718,2.005249,-0.135992
13,Isaac Smith,Geelong,8.839350,9.565567,-0.726218,65,4.0,2.0,32.0,0.662151,0.635777,0.432450,1.721962,2.091472,-0.296485
32,Robbie Fox,Sydney,7.458894,3.341723,4.117172,59,5.0,16.0,1.0,0.593856,0.294372,1.000000,1.334239,0.131580,2.514804


Player Ratings

In [16]:
players_vaep = get_player_vaep(exp_vaep_data)
players_vaep.sort_values('exp_vaep_value', ascending = False).head(5)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions
584,Marcus Bontempelli,Western Bulldogs,691.192864,717.905127,-26.712262,4145
149,Christian Petracca,Melbourne,657.065267,674.565748,-17.500480,4382
791,Shai Bolton,Richmond,589.869821,622.290809,-32.420989,2857
151,Clayton Oliver,Melbourne,535.705179,528.087770,7.617410,4264
319,Jack Macrae,Western Bulldogs,521.553076,516.903618,4.649456,4573


In [17]:
players_noshot_vaep = get_player_vaep(exp_vaep_data[exp_vaep_data['action_type'] != "Shot"])
players_noshot_vaep.sort_values('exp_vaep_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions
584,Marcus Bontempelli,Western Bulldogs,647.890166,659.717946,-11.827778,3995
149,Christian Petracca,Melbourne,575.787174,580.398092,-4.610917,4163
791,Shai Bolton,Richmond,527.929026,544.030224,-16.101199,2638
151,Clayton Oliver,Melbourne,518.700151,507.297310,11.402842,4205
319,Jack Macrae,Western Bulldogs,505.930566,497.695939,8.234625,4538
514,Lachie Neale,Brisbane Lions,494.481430,487.655909,6.825521,3923
701,Patrick Cripps,Carlton,470.502076,477.292964,-6.790890,3323
858,Tom Liberatore,Western Bulldogs,464.342393,435.734496,28.607896,3593
198,Darcy Parish,Essendon,460.772546,481.133840,-20.361294,3681
5,Aaron Naughton,Western Bulldogs,452.098900,469.216549,-17.117647,1467


Normalise for minutes played

In [18]:
player_minutes = get_player_minutes(player_stats)
players_vaep_mins = players_vaep.merge(player_minutes, how = 'left', on = "player")
player_ratings = convert_vaep_to_rating(players_vaep_mins)
player_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
584,Marcus Bontempelli,Western Bulldogs,691.192864,717.905127,-26.712262,4145,6008,71,11.504542,11.949153,-0.444612
791,Shai Bolton,Richmond,589.869821,622.290809,-32.420989,2857,5601,66,10.531509,11.110352,-0.578843
149,Christian Petracca,Melbourne,657.065267,674.565748,-17.500480,4382,6356,73,10.337717,10.613055,-0.275338
198,Darcy Parish,Essendon,475.525387,498.863440,-23.338054,3723,4636,57,10.257234,10.760644,-0.503409
151,Clayton Oliver,Melbourne,535.705179,528.087770,7.617410,4264,5374,62,9.968463,9.826717,0.141746
702,Patrick Dangerfield,Geelong,359.551695,380.794482,-21.242788,2392,3615,52,9.946105,10.533734,-0.587629
701,Patrick Cripps,Carlton,492.546811,504.477888,-11.931080,3409,5168,63,9.530705,9.761569,-0.230865
226,Dylan Shiel,Essendon,259.805741,275.710698,-15.904957,1702,2743,39,9.471591,10.051429,-0.579838
437,Joel Selwood,Geelong,298.174506,300.844638,-2.670132,2184,3233,44,9.222843,9.305433,-0.082590
856,Tom J. Lynch,Richmond,302.070005,331.813792,-29.743788,966,3282,37,9.203839,10.110109,-0.906270


In [19]:
player_minutes = get_player_minutes(player_stats)
players_noshot_vaep_mins = players_noshot_vaep.merge(player_minutes, how = 'left', on = "player")
player_noshot_ratings = convert_vaep_to_rating(players_noshot_vaep_mins)
player_noshot_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
584,Marcus Bontempelli,Western Bulldogs,647.890166,659.717946,-11.827778,3995,6008,71,10.783791,10.980658,-0.196867
198,Darcy Parish,Essendon,460.772546,481.133840,-20.361294,3681,4636,57,9.939011,10.378211,-0.439200
151,Clayton Oliver,Melbourne,518.700151,507.297310,11.402842,4205,5374,62,9.652031,9.439846,0.212185
791,Shai Bolton,Richmond,527.929026,544.030224,-16.101199,2638,5601,66,9.425621,9.713091,-0.287470
702,Patrick Dangerfield,Geelong,336.138970,353.275194,-17.136225,2319,3615,52,9.298450,9.772481,-0.474031
226,Dylan Shiel,Essendon,251.713549,264.773856,-13.060308,1679,2743,39,9.176579,9.652711,-0.476132
701,Patrick Cripps,Carlton,470.502076,477.292964,-6.790890,3323,5168,63,9.104142,9.235545,-0.131403
437,Joel Selwood,Geelong,293.464383,294.079618,-0.615235,2160,3233,44,9.077154,9.096184,-0.019030
149,Christian Petracca,Melbourne,575.787174,580.398092,-4.610917,4163,6356,73,9.058955,9.131499,-0.072544
856,Tom J. Lynch,Richmond,295.904042,310.183041,-14.278999,792,3282,37,9.015967,9.451037,-0.435070


In [20]:
player_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
584,Marcus Bontempelli,Western Bulldogs,691.192864,717.905127,-26.712262,4145,6008,71,11.504542,11.949153,-0.444612
791,Shai Bolton,Richmond,589.869821,622.290809,-32.420989,2857,5601,66,10.531509,11.110352,-0.578843
198,Darcy Parish,Essendon,475.525387,498.863440,-23.338054,3723,4636,57,10.257234,10.760644,-0.503409
149,Christian Petracca,Melbourne,657.065267,674.565748,-17.500480,4382,6356,73,10.337717,10.613055,-0.275338
702,Patrick Dangerfield,Geelong,359.551695,380.794482,-21.242788,2392,3615,52,9.946105,10.533734,-0.587629
856,Tom J. Lynch,Richmond,302.070005,331.813792,-29.743788,966,3282,37,9.203839,10.110109,-0.906270
226,Dylan Shiel,Essendon,259.805741,275.710698,-15.904957,1702,2743,39,9.471591,10.051429,-0.579838
151,Clayton Oliver,Melbourne,535.705179,528.087770,7.617410,4264,5374,62,9.968463,9.826717,0.141746
817,Taylor Walker,Adelaide,422.977849,467.130342,-44.152493,1592,4758,56,8.889824,9.817788,-0.927963
701,Patrick Cripps,Carlton,492.546811,504.477888,-11.931080,3409,5168,63,9.530705,9.761569,-0.230865


In [21]:
player_noshot_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
584,Marcus Bontempelli,Western Bulldogs,647.890166,659.717946,-11.827778,3995,6008,71,10.783791,10.980658,-0.196867
198,Darcy Parish,Essendon,460.772546,481.133840,-20.361294,3681,4636,57,9.939011,10.378211,-0.439200
702,Patrick Dangerfield,Geelong,336.138970,353.275194,-17.136225,2319,3615,52,9.298450,9.772481,-0.474031
791,Shai Bolton,Richmond,527.929026,544.030224,-16.101199,2638,5601,66,9.425621,9.713091,-0.287470
226,Dylan Shiel,Essendon,251.713549,264.773856,-13.060308,1679,2743,39,9.176579,9.652711,-0.476132
856,Tom J. Lynch,Richmond,295.904042,310.183041,-14.278999,792,3282,37,9.015967,9.451037,-0.435070
151,Clayton Oliver,Melbourne,518.700151,507.297310,11.402842,4205,5374,62,9.652031,9.439846,0.212185
701,Patrick Cripps,Carlton,470.502076,477.292964,-6.790890,3323,5168,63,9.104142,9.235545,-0.131403
149,Christian Petracca,Melbourne,575.787174,580.398092,-4.610917,4163,6356,73,9.058955,9.131499,-0.072544
437,Joel Selwood,Geelong,293.464383,294.079618,-0.615235,2160,3233,44,9.077154,9.096184,-0.019030


In [22]:
player_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
779,Sam Taylor,Greater Western Sydney,311.724511,90.010561,221.713950,2525,5400,57,5.772676,1.666862,4.105814
845,Tom Clurey,Port Adelaide,121.531071,36.617307,84.913763,810,2210,25,5.499143,1.656892,3.842252
264,Harris Andrews,Brisbane Lions,444.959790,187.737502,257.222288,3123,6907,71,6.442157,2.718076,3.724081
841,Tom Barrass,West Coast,262.407423,101.902847,160.504574,2084,4501,49,5.829981,2.264005,3.565976
757,Sam Collins,Gold Coast,338.729687,125.066150,213.663536,2416,6175,65,5.485501,2.025363,3.460138
24,Aliir Aliir,Port Adelaide,367.915880,157.159276,210.756605,2667,6363,66,5.782113,2.469893,3.312221
670,Nick Murray,Adelaide,195.967134,80.629001,115.338132,1208,3694,46,5.305012,2.182702,3.122310
349,Jacob Weitering,Carlton,351.530605,161.263436,190.267168,2747,6157,64,5.709446,2.619188,3.090258
197,Darcy Moore,Collingwood,349.117946,187.085960,162.031986,2691,5527,59,6.316590,3.384946,2.931644
427,Jeremy McGovern,West Coast,161.585082,86.102755,75.482326,1530,2619,34,6.169724,3.287620,2.882105


In [23]:
player_noshot_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
779,Sam Taylor,Greater Western Sydney,311.724511,90.010561,221.713950,2525,5400,57,5.772676,1.666862,4.105814
845,Tom Clurey,Port Adelaide,121.531071,36.617307,84.913763,810,2210,25,5.499143,1.656892,3.842252
264,Harris Andrews,Brisbane Lions,443.767324,186.199241,257.568083,3119,6907,71,6.424892,2.695805,3.729088
841,Tom Barrass,West Coast,262.274676,101.560790,160.713885,2083,4501,49,5.827031,2.256405,3.570626
757,Sam Collins,Gold Coast,338.729687,125.066150,213.663536,2416,6175,65,5.485501,2.025363,3.460138
24,Aliir Aliir,Port Adelaide,366.061504,155.452376,210.609129,2663,6363,66,5.752970,2.443067,3.309903
670,Nick Murray,Adelaide,194.987114,79.505134,115.481980,1204,3694,46,5.278482,2.152278,3.126204
349,Jacob Weitering,Carlton,348.422990,158.972615,189.450374,2745,6157,64,5.658973,2.581982,3.076992
197,Darcy Moore,Collingwood,350.386362,187.033723,163.352639,2682,5527,59,6.339540,3.384001,2.955539
427,Jeremy McGovern,West Coast,161.277148,85.476307,75.800840,1528,2619,34,6.157967,3.263700,2.894267


VAEP by Action

In [24]:
action_vaep = get_vaep_action_summary(exp_vaep_data)
action_vaep.sort_values("exp_vaep_value_per_action", ascending=False)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
3,Free For,17572.935779,14034.902478,3538.033318,23682,0.022803,0.742038,0.592640,0.149398,0.155103,0.126044,1.814862
1,Contested Mark,12717.786702,9645.264038,3072.522666,19542,0.018817,0.650792,0.493566,0.157227,0.112250,0.086622,1.576074
8,Knock On,2943.135126,2062.537970,880.597157,10982,0.010574,0.267996,0.187811,0.080185,0.025977,0.018523,0.451709
9,Loose Ball Get,24395.829504,17375.945398,7019.884101,111218,0.107089,0.219351,0.156233,0.063118,0.215323,0.156049,3.600904
10,Shot,6158.180416,8727.896610,-2569.716193,30328,0.029202,0.203053,0.287783,-0.084731,0.054354,0.078383,-1.318156
13,Uncontested Mark,21678.052316,21487.960630,190.091678,115543,0.111254,0.187619,0.185974,0.001645,0.191335,0.192978,0.097509
6,Hard Ball Get,7441.495656,4589.951637,2851.544015,52753,0.050795,0.141063,0.087008,0.054055,0.065680,0.041221,1.462722
4,Gather,5120.509868,4890.634380,229.875488,41244,0.039713,0.124152,0.118578,0.005574,0.045195,0.043922,0.117916
11,Spoil,4388.552157,-105.998356,4494.550512,41506,0.039965,0.105733,-0.002554,0.108287,0.038734,-0.000952,2.305515
0,Carry,13283.240465,16612.044026,-3328.803567,164195,0.158100,0.080899,0.101173,-0.020273,0.117241,0.149189,-1.707536


In [25]:
match_data = get_match(exp_vaep_data, match_id)
match_action_vaep = get_vaep_action_summary(match_data)
match_action_vaep.sort_values("exp_vaep_value_per_action", ascending=False)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
8,Knock On,10.797785,9.602597,1.195189,15,0.009288,0.719852,0.640173,0.079679,0.080737,0.064331,-0.076970
3,Free For,23.214205,18.527316,4.686889,35,0.021672,0.663263,0.529352,0.133911,0.173578,0.124122,-0.301836
1,Contested Mark,9.883480,7.997625,1.885855,16,0.009907,0.617717,0.499852,0.117866,0.073901,0.053579,-0.121449
10,Shot,9.150617,14.769121,-5.618505,47,0.029102,0.194694,0.314237,-0.119543,0.068421,0.098944,0.361832
9,Loose Ball Get,33.795576,24.548152,9.247424,190,0.117647,0.177871,0.129201,0.048671,0.252697,0.164457,-0.595535
12,Uncontested Mark,25.664626,25.998480,-0.333854,193,0.119505,0.132977,0.134707,-0.001730,0.191900,0.174174,0.021500
6,Hard Ball Get,6.997162,7.000760,-0.003597,80,0.049536,0.087465,0.087509,-0.000045,0.052319,0.046901,0.000232
7,Kick,22.121000,33.021796,-10.900796,374,0.231579,0.059147,0.088294,-0.029147,0.165404,0.221226,0.702012
4,Gather,1.368439,2.361971,-0.993532,59,0.036533,0.023194,0.040033,-0.016840,0.010232,0.015824,0.063984
0,Carry,5.624803,13.276728,-7.651924,265,0.164087,0.021226,0.050101,-0.028875,0.042058,0.088946,0.492784


In [26]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
9,Loose Ball Get,171.623087,150.367317,21.255769,511,0.116613,0.335857,0.294261,0.041596,0.261196,0.222910,-1.214582
4,Gather,106.661423,91.482006,15.179416,342,0.078047,0.311876,0.267491,0.044384,0.162330,0.135616,-0.867371
0,Carry,98.835962,103.415662,-4.579699,658,0.150160,0.150207,0.157167,-0.006960,0.150420,0.153307,0.261690
12,Uncontested Mark,93.488660,94.906350,-1.417690,333,0.075993,0.280747,0.285004,-0.004257,0.142282,0.140693,0.081009
10,Shot,81.278093,94.167657,-12.889564,219,0.049977,0.371133,0.429989,-0.058856,0.123699,0.139597,0.736526
3,Free For,48.105427,44.337935,3.767492,62,0.014149,0.775894,0.715128,0.060766,0.073213,0.065728,-0.215279
6,Hard Ball Get,47.991983,39.236503,8.755480,323,0.073711,0.148582,0.121475,0.027107,0.073040,0.058166,-0.500299
7,Kick,42.668578,68.666533,-25.997953,881,0.201050,0.048432,0.077942,-0.029510,0.064938,0.101794,1.485557
1,Contested Mark,13.051555,12.622731,0.428825,20,0.004564,0.652578,0.631137,0.021441,0.019863,0.018712,-0.024504
8,Knock On,3.071112,2.428423,0.642689,12,0.002738,0.255926,0.202369,0.053557,0.004674,0.003600,-0.036724


In [27]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,76.350840,91.827660,-15.476821,709,0.258006,0.107688,0.129517,-0.021829,0.348094,0.350775,0.364629
8,Loose Ball Get,57.596543,37.585477,20.011065,321,0.116812,0.179428,0.117089,0.062340,0.262590,0.143574,-0.471455
11,Uncontested Mark,33.247515,34.446619,-1.199104,194,0.070597,0.171379,0.177560,-0.006181,0.151580,0.131583,0.028251
2,Free For,31.656577,25.205364,6.451213,52,0.018923,0.608780,0.484719,0.124062,0.144327,0.096283,-0.151989
6,Kick,27.641643,47.106831,-19.465188,630,0.229258,0.043876,0.074773,-0.030897,0.126022,0.179945,0.458594
3,Gather,15.491058,11.377869,4.113190,103,0.037482,0.150399,0.110465,0.039934,0.070626,0.043463,-0.096906
9,Shot,15.116917,21.745405,-6.628488,56,0.020378,0.269945,0.388311,-0.118366,0.068920,0.083066,0.156165
5,Hard Ball Get,12.359773,9.495371,2.864402,77,0.028020,0.160517,0.123317,0.037200,0.056350,0.036272,-0.067484
7,Knock On,-0.160142,-0.530179,0.370037,6,0.002183,-0.026690,-0.088363,0.061673,-0.000730,-0.002025,-0.008718
10,Spoil,-1.037601,-0.130375,-0.907226,8,0.002911,-0.129700,-0.016297,-0.113403,-0.004731,-0.000498,0.021374


In [28]:
player = "Darcy Moore"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
11,Spoil,109.24,10.79,98.44,393,0.15,0.28,0.03,0.25,0.31,0.06,0.61
1,Contested Mark,105.46,30.95,74.51,205,0.08,0.51,0.15,0.36,0.30,0.17,0.46
12,Uncontested Mark,49.10,39.55,9.55,367,0.14,0.13,0.11,0.03,0.14,0.21,0.06
9,Loose Ball Get,38.50,10.30,28.20,215,0.08,0.18,0.05,0.13,0.11,0.06,0.17
3,Free For,33.51,14.80,18.71,42,0.02,0.80,0.35,0.45,0.10,0.08,0.12
6,Hard Ball Get,17.82,2.19,15.63,106,0.04,0.17,0.02,0.15,0.05,0.01,0.10
8,Knock On,12.60,4.48,8.13,61,0.02,0.21,0.07,0.13,0.04,0.02,0.05
4,Gather,3.27,3.98,-0.72,48,0.02,0.07,0.08,-0.01,0.01,0.02,-0.00
10,Shot,-1.27,0.05,-1.32,9,0.00,-0.14,0.01,-0.15,-0.00,0.00,-0.01
0,Carry,-2.09,26.33,-28.41,313,0.12,-0.01,0.08,-0.09,-0.01,0.14,-0.18


In [29]:
player = "Taylor Walker"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
1,Contested Mark,109.33,109.31,0.02,100,0.06,1.09,1.09,0.00,0.26,0.23,-0.00
12,Uncontested Mark,92.17,95.31,-3.14,206,0.13,0.45,0.46,-0.02,0.22,0.20,0.07
3,Free For,81.98,81.71,0.27,63,0.04,1.30,1.30,0.00,0.19,0.17,-0.01
10,Shot,59.18,93.17,-34.00,295,0.19,0.20,0.32,-0.12,0.14,0.20,0.77
9,Loose Ball Get,56.75,54.87,1.88,127,0.08,0.45,0.43,0.01,0.13,0.12,-0.04
7,Kick,37.29,39.90,-2.61,243,0.15,0.15,0.16,-0.01,0.09,0.09,0.06
6,Hard Ball Get,18.06,20.67,-2.61,114,0.07,0.16,0.18,-0.02,0.04,0.04,0.06
8,Knock On,12.60,11.91,0.69,20,0.01,0.63,0.60,0.03,0.03,0.03,-0.02
0,Carry,8.11,9.51,-1.41,124,0.08,0.07,0.08,-0.01,0.02,0.02,0.03
4,Gather,4.01,4.51,-0.50,45,0.03,0.09,0.10,-0.01,0.01,0.01,0.01


By Season

In [30]:
players_2021 = get_player_vaep(exp_vaep_2021)
players_2022 = get_player_vaep(exp_vaep_2022)
players_2023 = get_player_vaep(exp_vaep_2023)

In [31]:
player_minutes_2021 = get_player_minutes(player_stats_2021)
player_minutes_2022 = get_player_minutes(player_stats_2022)
player_minutes_2023 = get_player_minutes(player_stats_2023)

In [32]:
players_2021 = players_2021.merge(player_minutes_2021, how = "left", on = "player")
players_2022 = players_2022.merge(player_minutes_2022, how = "left", on = "player")
players_2023 = players_2023.merge(player_minutes_2023, how = "left", on = "player")

In [33]:
player_ratings_2021 = convert_vaep_to_rating(players_2021)
player_ratings_2022 = convert_vaep_to_rating(players_2022)
player_ratings_2023 = convert_vaep_to_rating(players_2023)

2021

In [34]:
player_ratings_2021.sort_values('exp_vaep_value', ascending = False).round(2).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
232,Jack Macrae,Western Bulldogs,239.83,243.36,-3.53,1872,2154,26,11.13,11.30,-0.16
110,Christian Petracca,Melbourne,234.27,248.30,-14.03,1524,2237,25,10.47,11.10,-0.63
112,Clayton Oliver,Melbourne,215.72,224.81,-9.10,1702,2211,25,9.76,10.17,-0.41
420,Marcus Bontempelli,Western Bulldogs,213.05,230.43,-17.38,1569,2193,26,9.72,10.51,-0.79
145,Darcy Parish,Essendon,206.24,214.38,-8.14,1492,1890,23,10.91,11.34,-0.43
154,Dayne Zorko,Brisbane Lions,202.00,203.43,-1.43,1199,1900,23,10.63,10.71,-0.08
618,Tom Liberatore,Western Bulldogs,187.01,185.16,1.85,1284,1967,25,9.51,9.41,0.09
100,Charlie Cameron,Brisbane Lions,184.22,198.37,-14.15,560,2082,24,8.85,9.53,-0.68
194,Harris Andrews,Brisbane Lions,174.72,61.69,113.04,1109,2224,23,7.86,2.77,5.08
268,Jake Stringer,Essendon,171.55,182.23,-10.68,661,1526,19,11.24,11.94,-0.70


In [35]:
player_ratings_2021.sort_values('vaep_rating', ascending = False).round(2).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
268,Jake Stringer,Essendon,171.55,182.23,-10.68,661,1526,19,11.24,11.94,-0.70
232,Jack Macrae,Western Bulldogs,239.83,243.36,-3.53,1872,2154,26,11.13,11.30,-0.16
145,Darcy Parish,Essendon,206.24,214.38,-8.14,1492,1890,23,10.91,11.34,-0.43
154,Dayne Zorko,Brisbane Lions,202.00,203.43,-1.43,1199,1900,23,10.63,10.71,-0.08
110,Christian Petracca,Melbourne,234.27,248.30,-14.03,1524,2237,25,10.47,11.10,-0.63
503,Patrick Dangerfield,Geelong,112.67,119.16,-6.48,749,1080,15,10.43,11.03,-0.60
567,Shai Bolton,Richmond,167.37,174.03,-6.66,847,1628,20,10.28,10.69,-0.41
42,Ben Cunnington,North Melbourne,126.23,125.10,1.13,880,1237,15,10.20,10.11,0.09
462,Nat Fyfe,Fremantle,123.59,120.21,3.38,790,1220,15,10.13,9.85,0.28
97,Chad Wingard,Hawthorn,123.48,130.64,-7.17,756,1239,16,9.97,10.54,-0.58


In [36]:
player_ratings_2021.sort_values('exp_offensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
110,Christian Petracca,Melbourne,234.271497,248.303884,-14.032386,1524,2237,25,10.472575,11.099861,-0.627286
232,Jack Macrae,Western Bulldogs,239.833772,243.362010,-3.528238,1872,2154,26,11.134344,11.298143,-0.163799
420,Marcus Bontempelli,Western Bulldogs,213.050205,230.433339,-17.383133,1569,2193,26,9.715012,10.507676,-0.792665
112,Clayton Oliver,Melbourne,215.717201,224.814796,-9.097594,1702,2211,25,9.756545,10.168014,-0.411470
145,Darcy Parish,Essendon,206.237722,214.380642,-8.142920,1492,1890,23,10.912049,11.342891,-0.430842
154,Dayne Zorko,Brisbane Lions,202.004690,203.434612,-1.429922,1199,1900,23,10.631826,10.707085,-0.075259
100,Charlie Cameron,Brisbane Lions,184.223370,198.370906,-14.147538,560,2082,24,8.848385,9.527901,-0.679517
618,Tom Liberatore,Western Bulldogs,187.010501,185.158661,1.851841,1284,1967,25,9.507397,9.413252,0.094145
268,Jake Stringer,Essendon,171.548902,182.227177,-10.678275,661,1526,19,11.241737,11.941493,-0.699756
312,Joe Daniher,Brisbane Lions,170.090351,181.322744,-11.232392,742,2252,24,7.552857,8.051632,-0.498774


In [37]:
player_ratings_2021.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
268,Jake Stringer,Essendon,171.548902,182.227177,-10.678275,661,1526,19,11.241737,11.941493,-0.699756
145,Darcy Parish,Essendon,206.237722,214.380642,-8.142920,1492,1890,23,10.912049,11.342891,-0.430842
232,Jack Macrae,Western Bulldogs,239.833772,243.362010,-3.528238,1872,2154,26,11.134344,11.298143,-0.163799
110,Christian Petracca,Melbourne,234.271497,248.303884,-14.032386,1524,2237,25,10.472575,11.099861,-0.627286
503,Patrick Dangerfield,Geelong,112.673279,119.156139,-6.482860,749,1080,15,10.432711,11.032976,-0.600265
154,Dayne Zorko,Brisbane Lions,202.004690,203.434612,-1.429922,1199,1900,23,10.631826,10.707085,-0.075259
567,Shai Bolton,Richmond,167.365853,174.025674,-6.659821,847,1628,20,10.280458,10.689538,-0.409080
97,Chad Wingard,Hawthorn,123.476775,130.643899,-7.167124,756,1239,16,9.965841,10.544302,-0.578460
420,Marcus Bontempelli,Western Bulldogs,213.050205,230.433339,-17.383133,1569,2193,26,9.715012,10.507676,-0.792665
589,Taylor Walker,Adelaide,130.613395,142.150363,-11.536967,441,1369,16,9.540789,10.383518,-0.842730


In [38]:
player_ratings_2021.sort_values('exp_defensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
194,Harris Andrews,Brisbane Lions,174.721199,61.685687,113.035510,1109,2224,23,7.856169,2.773637,5.082532
18,Aliir Aliir,Port Adelaide,148.892637,70.819226,78.073411,1042,2315,24,6.431647,3.059146,3.372502
391,Liam Jones,Carlton,125.419631,50.343471,75.076161,716,1814,19,6.913982,2.775274,4.138708
540,Sam Collins,Gold Coast,102.418424,31.083071,71.335354,725,1788,20,5.728100,1.738427,3.989673
557,Sam Taylor,Greater Western Sydney,105.751347,34.665822,71.085525,829,1781,19,5.937751,1.946425,3.991326
13,Alex Keath,Western Bulldogs,112.246191,42.624855,69.621337,803,2173,23,5.165494,1.961567,3.203927
253,Jacob Weitering,Carlton,137.419024,68.449473,68.969551,1063,2110,22,6.512750,3.244051,3.268699
263,Jake Lever,Melbourne,146.651849,78.215343,68.436506,1141,2472,25,5.932518,3.164051,2.768467
617,Tom Jonas,Port Adelaide,112.552557,46.355123,66.197434,985,2297,24,4.899981,2.018072,2.881908
163,Dougal Howard,St Kilda,94.079411,41.332038,52.747374,935,1983,21,4.744297,2.084319,2.659979


In [39]:
player_ratings_2021.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
194,Harris Andrews,Brisbane Lions,174.721199,61.685687,113.035510,1109,2224,23,7.856169,2.773637,5.082532
391,Liam Jones,Carlton,125.419631,50.343471,75.076161,716,1814,19,6.913982,2.775274,4.138708
557,Sam Taylor,Greater Western Sydney,105.751347,34.665822,71.085525,829,1781,19,5.937751,1.946425,3.991326
540,Sam Collins,Gold Coast,102.418424,31.083071,71.335354,725,1788,20,5.728100,1.738427,3.989673
18,Aliir Aliir,Port Adelaide,148.892637,70.819226,78.073411,1042,2315,24,6.431647,3.059146,3.372502
308,Jeremy McGovern,West Coast,82.370477,42.766708,39.603769,672,1204,15,6.841402,3.552052,3.289350
253,Jacob Weitering,Carlton,137.419024,68.449473,68.969551,1063,2110,22,6.512750,3.244051,3.268699
13,Alex Keath,Western Bulldogs,112.246191,42.624855,69.621337,803,2173,23,5.165494,1.961567,3.203927
73,Brennan Cox,Fremantle,55.475010,21.306055,34.168955,459,1085,12,5.112904,1.963692,3.149212
196,Harrison Petty,Melbourne,74.524783,22.932792,51.591991,511,1682,19,4.430724,1.363424,3.067300


2022

In [40]:
player_ratings_2022.sort_values('exp_vaep_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
582,Shai Bolton,Richmond,219.539253,239.618246,-20.078994,903,1985,23,11.059912,12.071448,-1.011536
432,Marcus Bontempelli,Western Bulldogs,212.899263,220.691022,-7.791758,1158,1863,22,11.427765,11.846002,-0.418237
110,Christian Petracca,Melbourne,208.061993,204.705501,3.356492,1416,2108,24,9.870114,9.710887,0.159226
112,Clayton Oliver,Melbourne,204.197155,192.967085,11.230070,1629,1995,23,10.235446,9.672536,0.562911
383,Lachie Neale,Brisbane Lions,197.983585,204.705531,-6.721946,1621,2182,25,9.073492,9.381555,-0.308064
514,Patrick Cripps,Carlton,191.837921,199.463357,-7.625438,1252,1707,21,11.238308,11.685024,-0.446716
97,Chad Warner,Sydney,189.178691,208.473954,-19.295263,1208,1938,24,9.761542,10.757170,-0.995628
638,Touk Miller,Gold Coast,188.795334,211.827466,-23.032131,1320,1907,22,9.900122,11.107890,-1.207768
104,Charlie Curnow,Carlton,185.580779,203.896541,-18.315761,574,2022,22,9.178080,10.083904,-0.905824
626,Tom J. Lynch,Richmond,183.969058,201.725794,-17.756737,514,1601,19,11.490884,12.599987,-1.109103


In [41]:
player_ratings_2022.sort_values('vaep_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
626,Tom J. Lynch,Richmond,183.969058,201.725794,-17.756737,514,1601,19,11.490884,12.599987,-1.109103
432,Marcus Bontempelli,Western Bulldogs,212.899263,220.691022,-7.791758,1158,1863,22,11.427765,11.846002,-0.418237
514,Patrick Cripps,Carlton,191.837921,199.463357,-7.625438,1252,1707,21,11.238308,11.685024,-0.446716
582,Shai Bolton,Richmond,219.539253,239.618246,-20.078994,903,1985,23,11.059912,12.071448,-1.011536
145,Darcy Parish,Essendon,135.627070,140.660201,-5.033132,1061,1299,16,10.440883,10.828345,-0.387462
164,Dylan Shiel,Essendon,146.396585,157.286773,-10.890187,923,1404,19,10.427107,11.202762,-0.775654
112,Clayton Oliver,Melbourne,204.197155,192.967085,11.230070,1629,1995,23,10.235446,9.672536,0.562911
93,Cam Rayner,Brisbane Lions,179.780534,174.631974,5.148560,756,1779,24,10.105707,9.816300,0.289408
638,Touk Miller,Gold Coast,188.795334,211.827466,-23.032131,1320,1907,22,9.900122,11.107890,-1.207768
110,Christian Petracca,Melbourne,208.061993,204.705501,3.356492,1416,2108,24,9.870114,9.710887,0.159226


In [42]:
player_ratings_2022.sort_values('exp_offensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
582,Shai Bolton,Richmond,219.539253,239.618246,-20.078994,903,1985,23,11.059912,12.071448,-1.011536
432,Marcus Bontempelli,Western Bulldogs,212.899263,220.691022,-7.791758,1158,1863,22,11.427765,11.846002,-0.418237
638,Touk Miller,Gold Coast,188.795334,211.827466,-23.032131,1320,1907,22,9.900122,11.107890,-1.207768
97,Chad Warner,Sydney,189.178691,208.473954,-19.295263,1208,1938,24,9.761542,10.757170,-0.995628
383,Lachie Neale,Brisbane Lions,197.983585,204.705531,-6.721946,1621,2182,25,9.073492,9.381555,-0.308064
110,Christian Petracca,Melbourne,208.061993,204.705501,3.356492,1416,2108,24,9.870114,9.710887,0.159226
104,Charlie Curnow,Carlton,185.580779,203.896541,-18.315761,574,2022,22,9.178080,10.083904,-0.905824
626,Tom J. Lynch,Richmond,183.969058,201.725794,-17.756737,514,1601,19,11.490884,12.599987,-1.109103
514,Patrick Cripps,Carlton,191.837921,199.463357,-7.625438,1252,1707,21,11.238308,11.685024,-0.446716
112,Clayton Oliver,Melbourne,204.197155,192.967085,11.230070,1629,1995,23,10.235446,9.672536,0.562911


In [43]:
player_ratings_2022.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
626,Tom J. Lynch,Richmond,183.969058,201.725794,-17.756737,514,1601,19,11.490884,12.599987,-1.109103
582,Shai Bolton,Richmond,219.539253,239.618246,-20.078994,903,1985,23,11.059912,12.071448,-1.011536
432,Marcus Bontempelli,Western Bulldogs,212.899263,220.691022,-7.791758,1158,1863,22,11.427765,11.846002,-0.418237
514,Patrick Cripps,Carlton,191.837921,199.463357,-7.625438,1252,1707,21,11.238308,11.685024,-0.446716
164,Dylan Shiel,Essendon,146.396585,157.286773,-10.890187,923,1404,19,10.427107,11.202762,-0.775654
638,Touk Miller,Gold Coast,188.795334,211.827466,-23.032131,1320,1907,22,9.900122,11.107890,-1.207768
145,Darcy Parish,Essendon,135.627070,140.660201,-5.033132,1061,1299,16,10.440883,10.828345,-0.387462
97,Chad Warner,Sydney,189.178691,208.473954,-19.295263,1208,1938,24,9.761542,10.757170,-0.995628
104,Charlie Curnow,Carlton,185.580779,203.896541,-18.315761,574,2022,22,9.178080,10.083904,-0.905824
330,Jordan De Goey,Collingwood,150.286312,164.288507,-14.002195,854,1643,19,9.147067,9.999300,-0.852233


In [44]:
player_ratings_2022.sort_values('exp_defensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
144,Darcy Moore,Collingwood,149.558812,68.188295,81.370516,1044,2314,24,6.463216,2.946772,3.516444
573,Sam Taylor,Greater Western Sydney,104.821076,26.820952,78.000125,1018,2132,22,4.916561,1.258018,3.658542
615,Tom Barrass,West Coast,108.013898,38.692133,69.321765,865,1767,19,6.112841,2.189708,3.923133
555,Sam Collins,Gold Coast,111.841719,45.629009,66.212711,798,2154,22,5.192280,2.118338,3.073942
188,Harris Andrews,Brisbane Lions,109.400661,44.330955,65.069705,948,2337,24,4.681244,1.896917,2.784326
558,Sam De Koning,Geelong,88.986176,33.113386,55.872790,825,2114,23,4.209374,1.566385,2.642989
551,Ryan Gardner,Western Bulldogs,88.843603,33.456009,55.387594,779,2152,22,4.128420,1.554647,2.573773
592,Steven May,Melbourne,118.375820,64.444097,53.931722,1106,2049,22,5.777248,3.145149,2.632100
286,James Sicily,Hawthorn,140.888284,88.453104,52.435180,1363,2110,22,6.677170,4.192090,2.485080
255,Jacob Weitering,Carlton,83.368010,31.723213,51.644797,644,1680,18,4.962382,1.888286,3.074095


In [45]:
player_ratings_2022.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
615,Tom Barrass,West Coast,108.013898,38.692133,69.321765,865,1767,19,6.112841,2.189708,3.923133
617,Tom Clurey,Port Adelaide,62.331676,17.552252,44.779423,392,1182,13,5.273407,1.484962,3.788445
573,Sam Taylor,Greater Western Sydney,104.821076,26.820952,78.000125,1018,2132,22,4.916561,1.258018,3.658542
144,Darcy Moore,Collingwood,149.558812,68.188295,81.370516,1044,2314,24,6.463216,2.946772,3.516444
162,Dylan Grimes,Richmond,57.543975,18.813341,38.730634,481,1228,15,4.685991,1.532031,3.153960
255,Jacob Weitering,Carlton,83.368010,31.723213,51.644797,644,1680,18,4.962382,1.888286,3.074095
555,Sam Collins,Gold Coast,111.841719,45.629009,66.212711,798,2154,22,5.192280,2.118338,3.073942
540,Robbie Tarrant,Richmond,77.346181,26.074010,51.272172,615,1829,20,4.228878,1.425588,2.803290
188,Harris Andrews,Brisbane Lions,109.400661,44.330955,65.069705,948,2337,24,4.681244,1.896917,2.784326
558,Sam De Koning,Geelong,88.986176,33.113386,55.872790,825,2114,23,4.209374,1.566385,2.642989


2023

In [46]:
player_ratings_2023.sort_values('exp_vaep_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
424,Marcus Bontempelli,Western Bulldogs,265.243396,266.780767,-1.537371,1418,1952,23,13.588289,13.667047,-0.078759
490,Noah Anderson,Gold Coast,218.872322,211.209966,7.662354,1352,1941,23,11.276266,10.881503,0.394763
111,Christian Petracca,Melbourne,214.731777,221.556364,-6.824586,1442,2011,24,10.677861,11.017223,-0.339363
660,Zak Butters,Port Adelaide,209.754777,212.579749,-2.824972,1467,1879,24,11.163107,11.313451,-0.150344
573,Shai Bolton,Richmond,202.964715,208.646889,-5.682173,1107,1988,23,10.209493,10.495316,-0.285824
105,Charlie Curnow,Carlton,200.583676,217.557909,-16.974233,733,2246,24,8.930707,9.686461,-0.755754
132,Dan Houston,Port Adelaide,183.020963,164.756446,18.264517,1301,1974,24,9.271579,8.346325,0.925254
376,Lachie Neale,Brisbane Lions,180.108747,176.162362,3.946385,1394,2077,24,8.671581,8.481577,0.190004
329,Jordan Dawson,Adelaide,175.908603,180.559147,-4.650545,1372,1978,23,8.893256,9.128369,-0.235113
578,Stephen Coniglio,Greater Western Sydney,173.999626,191.705699,-17.706074,1354,1893,23,9.191739,10.127084,-0.935345


In [47]:
player_ratings_2023.sort_values('vaep_rating', ascending = False).round(2).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
424,Marcus Bontempelli,Western Bulldogs,265.24,266.78,-1.54,1418,1952,23,13.59,13.67,-0.08
490,Noah Anderson,Gold Coast,218.87,211.21,7.66,1352,1941,23,11.28,10.88,0.39
660,Zak Butters,Port Adelaide,209.75,212.58,-2.82,1467,1879,24,11.16,11.31,-0.15
111,Christian Petracca,Melbourne,214.73,221.56,-6.82,1442,2011,24,10.68,11.02,-0.34
573,Shai Bolton,Richmond,202.96,208.65,-5.68,1107,1988,23,10.21,10.50,-0.29
113,Clayton Oliver,Melbourne,115.79,110.31,5.48,933,1168,14,9.91,9.44,0.47
510,Patrick Dangerfield,Geelong,116.09,131.29,-15.20,781,1191,18,9.75,11.02,-1.28
509,Patrick Cripps,Carlton,169.75,166.71,3.04,1159,1781,22,9.53,9.36,0.17
409,Luke Davies-Uniacke,North Melbourne,106.70,111.89,-5.19,826,1133,15,9.42,9.88,-0.46
132,Dan Houston,Port Adelaide,183.02,164.76,18.26,1301,1974,24,9.27,8.35,0.93


In [48]:
player_ratings_2023.sort_values('exp_offensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
424,Marcus Bontempelli,Western Bulldogs,265.243396,266.780767,-1.537371,1418,1952,23,13.588289,13.667047,-0.078759
111,Christian Petracca,Melbourne,214.731777,221.556364,-6.824586,1442,2011,24,10.677861,11.017223,-0.339363
105,Charlie Curnow,Carlton,200.583676,217.557909,-16.974233,733,2246,24,8.930707,9.686461,-0.755754
660,Zak Butters,Port Adelaide,209.754777,212.579749,-2.824972,1467,1879,24,11.163107,11.313451,-0.150344
490,Noah Anderson,Gold Coast,218.872322,211.209966,7.662354,1352,1941,23,11.276266,10.881503,0.394763
573,Shai Bolton,Richmond,202.964715,208.646889,-5.682173,1107,1988,23,10.209493,10.495316,-0.285824
578,Stephen Coniglio,Greater Western Sydney,173.999626,191.705699,-17.706074,1354,1893,23,9.191739,10.127084,-0.935345
118,Connor Rozee,Port Adelaide,171.456321,187.894370,-16.438049,1335,2012,24,8.521686,9.338686,-0.817000
585,Taylor Walker,Adelaide,166.380501,187.621823,-21.241323,598,1823,22,9.126742,10.291927,-1.165185
329,Jordan Dawson,Adelaide,175.908603,180.559147,-4.650545,1372,1978,23,8.893256,9.128369,-0.235113


In [49]:
player_ratings_2023.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
424,Marcus Bontempelli,Western Bulldogs,265.243396,266.780767,-1.537371,1418,1952,23,13.588289,13.667047,-0.078759
660,Zak Butters,Port Adelaide,209.754777,212.579749,-2.824972,1467,1879,24,11.163107,11.313451,-0.150344
510,Patrick Dangerfield,Geelong,116.089333,131.292226,-15.202893,781,1191,18,9.747215,11.023697,-1.276481
111,Christian Petracca,Melbourne,214.731777,221.556364,-6.824586,1442,2011,24,10.677861,11.017223,-0.339363
490,Noah Anderson,Gold Coast,218.872322,211.209966,7.662354,1352,1941,23,11.276266,10.881503,0.394763
573,Shai Bolton,Richmond,202.964715,208.646889,-5.682173,1107,1988,23,10.209493,10.495316,-0.285824
585,Taylor Walker,Adelaide,166.380501,187.621823,-21.241323,598,1823,22,9.126742,10.291927,-1.165185
578,Stephen Coniglio,Greater Western Sydney,173.999626,191.705699,-17.706074,1354,1893,23,9.191739,10.127084,-0.935345
148,Darcy Parish,Essendon,133.660595,143.822596,-10.162001,1170,1447,18,9.237083,9.939364,-0.702281
409,Luke Davies-Uniacke,North Melbourne,106.702537,111.887653,-5.185116,826,1133,15,9.417700,9.875344,-0.457645


In [50]:
player_ratings_2023.sort_values('exp_defensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
226,Jack Buckley,Greater Western Sydney,142.481308,56.577805,85.903502,961,2232,24,6.383571,2.534848,3.848723
18,Aliir Aliir,Port Adelaide,136.143998,50.551270,85.592728,869,2251,23,6.048156,2.245725,3.802431
193,Harris Andrews,Brisbane Lions,160.837931,81.720859,79.117073,1066,2346,24,6.855837,3.483413,3.372424
546,Sam Collins,Gold Coast,124.469544,48.354071,76.115472,893,2233,23,5.574095,2.165431,3.408664
487,Nick Murray,Adelaide,108.773244,35.912750,72.860494,520,1353,17,8.039412,2.654305,5.385107
562,Sam Taylor,Greater Western Sydney,101.152087,28.523787,72.628300,678,1487,16,6.802427,1.918210,4.884217
261,Jacob Weitering,Carlton,130.743571,61.090750,69.652820,1040,2367,24,5.523598,2.580936,2.942662
101,Charlie Ballard,Gold Coast,112.622342,51.381278,61.241064,861,2202,23,5.114548,2.333391,2.781156
603,Tom Barrass,West Coast,83.891314,27.002777,56.888536,572,1341,14,6.255877,2.013630,4.242247
616,Tom McCartin,Sydney,88.372621,31.644505,56.728116,607,1310,15,6.746002,2.415611,4.330391


In [51]:
player_ratings_2023.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
487,Nick Murray,Adelaide,108.773244,35.912750,72.860494,520,1353,17,8.039412,2.654305,5.385107
562,Sam Taylor,Greater Western Sydney,101.152087,28.523787,72.628300,678,1487,16,6.802427,1.918210,4.884217
616,Tom McCartin,Sydney,88.372621,31.644505,56.728116,607,1310,15,6.746002,2.415611,4.330391
603,Tom Barrass,West Coast,83.891314,27.002777,56.888536,572,1341,14,6.255877,2.013630,4.242247
226,Jack Buckley,Greater Western Sydney,142.481308,56.577805,85.903502,961,2232,24,6.383571,2.534848,3.848723
18,Aliir Aliir,Port Adelaide,136.143998,50.551270,85.592728,869,2251,23,6.048156,2.245725,3.802431
546,Sam Collins,Gold Coast,124.469544,48.354071,76.115472,893,2233,23,5.574095,2.165431,3.408664
193,Harris Andrews,Brisbane Lions,160.837931,81.720859,79.117073,1066,2346,24,6.855837,3.483413,3.372424
261,Jacob Weitering,Carlton,130.743571,61.090750,69.652820,1040,2367,24,5.523598,2.580936,2.942662
101,Charlie Ballard,Gold Coast,112.622342,51.381278,61.241064,861,2202,23,5.114548,2.333391,2.781156


In [52]:
action_vaep = get_vaep_action_summary(exp_vaep_2021)
action_vaep.sort_values("exp_vaep_value_per_action")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-715.613493,-422.608254,-293.005240,972,0.002816,-0.736228,-0.434782,-0.301446,-0.019284,-0.011648,-0.354050
12,Tackle,-9.170337,1.808461,-10.978797,129,0.000374,-0.071088,0.014019,-0.085107,-0.000247,0.000050,-0.013266
5,Handball,-2826.930566,-1047.020262,-1779.910303,60799,0.176132,-0.046496,-0.017221,-0.029275,-0.076179,-0.028858,-2.150738
7,Kick,3143.340782,5598.894487,-2455.553707,80643,0.233619,0.038978,0.069428,-0.030450,0.084706,0.154319,-2.967146
0,Carry,4437.644371,5540.343549,-1102.699176,54634,0.158272,0.081225,0.101408,-0.020183,0.119584,0.152705,-1.332437
11,Spoil,1398.923576,-43.572206,1442.495778,13590,0.039370,0.102938,-0.003206,0.106144,0.037698,-0.001201,1.743027
4,Gather,1577.770200,1546.773536,30.996664,14029,0.040641,0.112465,0.110255,0.002209,0.042517,0.042633,0.037455
6,Hard Ball Get,2539.341442,1530.061872,1009.279566,18947,0.054889,0.134023,0.080755,0.053269,0.068429,0.042172,1.219554
13,Uncontested Mark,6827.745847,6813.562453,14.183393,38669,0.112022,0.176569,0.176202,0.000367,0.183992,0.187798,0.017138
10,Shot,2033.469495,2851.155774,-817.686278,9834,0.028489,0.206779,0.289928,-0.083149,0.054797,0.078585,-0.988044


In [53]:
action_vaep = get_vaep_action_summary(exp_vaep_2022)
action_vaep.sort_values("exp_vaep_value_per_action")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-893.667639,-525.264185,-368.403454,1198,0.003503,-0.745966,-0.438451,-0.307515,-0.023779,-0.014209,-0.598542
12,Tackle,-24.815948,-12.282417,-12.533531,84,0.000246,-0.295428,-0.146219,-0.149209,-0.000660,-0.000332,-0.020363
5,Handball,-2812.762001,-945.367936,-1867.394065,59270,0.173288,-0.047457,-0.015950,-0.031507,-0.074843,-0.025573,-3.033939
7,Kick,2903.108385,5450.367207,-2547.258827,79965,0.233794,0.036305,0.068159,-0.031855,0.077246,0.147439,-4.138509
0,Carry,4075.753176,5218.420918,-1142.667747,53272,0.155752,0.076508,0.097958,-0.021450,0.108448,0.141165,-1.856482
11,Spoil,1501.866075,-18.951876,1520.817950,13964,0.040827,0.107553,-0.001357,0.108910,0.039962,-0.000513,2.470860
4,Gather,1773.598895,1665.008057,108.590838,13390,0.039148,0.132457,0.124347,0.008110,0.047192,0.045041,0.176427
6,Hard Ball Get,2479.627586,1544.033553,935.594034,17178,0.050223,0.144349,0.089884,0.054465,0.065978,0.041768,1.520051
13,Uncontested Mark,7186.999099,7115.073383,71.925715,38101,0.111396,0.188630,0.186742,0.001888,0.191233,0.192471,0.116857
10,Shot,2051.695930,2925.983822,-874.287893,10069,0.029439,0.203764,0.290593,-0.086830,0.054592,0.079151,-1.420448


In [54]:
action_vaep = get_vaep_action_summary(exp_vaep_2023)
action_vaep.sort_values("exp_vaep_value_per_action")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-791.550716,-458.534489,-333.016229,1093,0.003111,-0.724200,-0.419519,-0.304681,-0.020503,-0.012035,-0.657621
12,Tackle,-15.348031,-6.400358,-8.947673,103,0.000293,-0.149010,-0.062139,-0.086871,-0.000398,-0.000168,-0.017669
5,Handball,-3453.195087,-1419.239308,-2033.955776,62735,0.178564,-0.055044,-0.022623,-0.032421,-0.089444,-0.037249,-4.016535
7,Kick,3095.556127,5813.699496,-2718.143361,80569,0.229325,0.038421,0.072158,-0.033737,0.080181,0.152587,-5.367628
0,Carry,4769.842919,5853.279559,-1083.436644,56289,0.160216,0.084738,0.103986,-0.019248,0.123548,0.153626,-2.139506
11,Spoil,1487.762507,-43.474275,1531.236784,13952,0.039712,0.106634,-0.003116,0.109750,0.038536,-0.001141,3.023796
4,Gather,1769.140773,1678.852788,90.287986,13825,0.039350,0.127967,0.121436,0.006531,0.045824,0.044063,0.178295
6,Hard Ball Get,2422.526628,1515.856211,906.670415,16628,0.047329,0.145690,0.091163,0.054527,0.062748,0.039785,1.790439
13,Uncontested Mark,7663.307371,7559.324794,103.982571,38773,0.110360,0.197645,0.194964,0.002682,0.198494,0.198403,0.205339
10,Shot,2073.014991,2950.757014,-877.742023,10425,0.029673,0.198850,0.283046,-0.084196,0.053695,0.077446,-1.733313


In [55]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,29.56,40.61,-11.05,361,0.25,0.08,0.11,-0.03,0.37,0.36,0.32
8,Loose Ball Get,25.30,18.52,6.78,163,0.11,0.16,0.11,0.04,0.32,0.16,-0.20
11,Uncontested Mark,17.84,18.54,-0.70,124,0.09,0.14,0.15,-0.01,0.23,0.16,0.02
2,Free For,14.21,10.49,3.73,28,0.02,0.51,0.37,0.13,0.18,0.09,-0.11
6,Kick,13.31,27.34,-14.03,344,0.24,0.04,0.08,-0.04,0.17,0.24,0.41
5,Hard Ball Get,4.25,1.61,2.64,47,0.03,0.09,0.03,0.06,0.05,0.01,-0.08
9,Shot,3.65,5.57,-1.92,17,0.01,0.21,0.33,-0.11,0.05,0.05,0.06
3,Gather,1.93,1.72,0.21,44,0.03,0.04,0.04,0.00,0.02,0.02,-0.01
7,Knock On,0.43,0.10,0.33,4,0.00,0.11,0.03,0.08,0.01,0.00,-0.01
10,Spoil,0.16,0.32,-0.15,5,0.00,0.03,0.06,-0.03,0.00,0.00,0.00


In [56]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,46.79,51.21,-4.42,348,0.26,0.13,0.15,-0.01,0.33,0.35,0.56
8,Loose Ball Get,32.30,19.06,13.24,158,0.12,0.20,0.12,0.08,0.23,0.13,-1.67
2,Free For,17.44,14.72,2.73,24,0.02,0.73,0.61,0.11,0.12,0.10,-0.34
11,Uncontested Mark,15.41,15.91,-0.50,70,0.05,0.22,0.23,-0.01,0.11,0.11,0.06
6,Kick,14.33,19.77,-5.44,286,0.22,0.05,0.07,-0.02,0.10,0.13,0.69
3,Gather,13.56,9.65,3.91,59,0.04,0.23,0.16,0.07,0.10,0.07,-0.49
9,Shot,11.47,16.17,-4.71,39,0.03,0.29,0.41,-0.12,0.08,0.11,0.60
5,Hard Ball Get,8.11,7.89,0.22,30,0.02,0.27,0.26,0.01,0.06,0.05,-0.03
7,Knock On,-0.59,-0.63,0.04,2,0.00,-0.30,-0.32,0.02,-0.00,-0.00,-0.01
1,Error,-0.98,-0.51,-0.47,3,0.00,-0.33,-0.17,-0.16,-0.01,-0.00,0.06


In [57]:
player_ratings_2023[player_ratings_2023['player'] == 'Nick Daicos']

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
482,Nick Daicos,Collingwood,140.367724,148.272501,-7.904777,1322,1689,20,8.3107,8.778715,-0.468015


In [58]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
9,Loose Ball Get,51.14,44.03,7.11,157,0.11,0.33,0.28,0.05,0.25,0.22,2.12
0,Carry,33.70,35.66,-1.96,232,0.16,0.15,0.15,-0.01,0.16,0.17,-0.58
4,Gather,33.55,27.70,5.84,112,0.08,0.30,0.25,0.05,0.16,0.14,1.74
10,Shot,27.52,30.60,-3.07,69,0.05,0.40,0.44,-0.04,0.13,0.15,-0.92
6,Hard Ball Get,21.24,16.27,4.97,102,0.07,0.21,0.16,0.05,0.10,0.08,1.48
12,Uncontested Mark,21.06,20.78,0.28,103,0.07,0.20,0.20,0.00,0.10,0.10,0.08
7,Kick,14.28,19.14,-4.86,288,0.20,0.05,0.07,-0.02,0.07,0.09,-1.45
3,Free For,11.95,10.74,1.21,16,0.01,0.75,0.67,0.08,0.06,0.05,0.36
1,Contested Mark,3.18,3.19,-0.02,4,0.00,0.79,0.80,-0.00,0.02,0.02,-0.00
11,Spoil,0.97,-0.05,1.02,5,0.00,0.19,-0.01,0.20,0.00,-0.00,0.30


In [59]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
9,Loose Ball Get,67.92,58.84,9.08,194,0.13,0.35,0.30,0.05,0.32,0.27,-1.33
4,Gather,30.20,28.29,1.90,109,0.08,0.28,0.26,0.02,0.14,0.13,-0.28
12,Uncontested Mark,29.76,31.18,-1.42,103,0.07,0.29,0.30,-0.01,0.14,0.14,0.21
10,Shot,27.43,31.52,-4.09,81,0.06,0.34,0.39,-0.05,0.13,0.14,0.60
0,Carry,26.95,28.22,-1.28,203,0.14,0.13,0.14,-0.01,0.13,0.13,0.19
3,Free For,19.18,17.60,1.58,23,0.02,0.83,0.77,0.07,0.09,0.08,-0.23
7,Kick,17.82,25.88,-8.06,253,0.18,0.07,0.10,-0.03,0.08,0.12,1.18
6,Hard Ball Get,11.02,7.58,3.44,110,0.08,0.10,0.07,0.03,0.05,0.03,-0.50
1,Contested Mark,7.98,7.80,0.17,7,0.00,1.14,1.11,0.02,0.04,0.04,-0.03
8,Knock On,2.02,1.35,0.68,2,0.00,1.01,0.67,0.34,0.01,0.01,-0.10


In [60]:
player_ratings_2023[player_ratings_2023['player'] ==  "Christian Petracca"]

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
111,Christian Petracca,Melbourne,214.731777,221.556364,-6.824586,1442,2011,24,10.677861,11.017223,-0.339363


In [61]:
team_data = get_team(exp_vaep_2022, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
9,Loose Ball Get,551.56,403.53,148.03,2255,0.11,0.24,0.18,0.07,0.22,0.17,1.70
13,Uncontested Mark,504.46,508.15,-3.68,2481,0.12,0.20,0.20,-0.00,0.20,0.21,-0.04
3,Free For,348.02,292.47,55.56,497,0.02,0.70,0.59,0.11,0.14,0.12,0.64
0,Carry,310.85,369.57,-58.72,3308,0.16,0.09,0.11,-0.02,0.12,0.15,-0.67
7,Kick,258.71,345.67,-86.96,4833,0.23,0.05,0.07,-0.02,0.10,0.14,-1.00
1,Contested Mark,248.45,189.04,59.41,387,0.02,0.64,0.49,0.15,0.10,0.08,0.68
6,Hard Ball Get,156.77,95.38,61.39,1042,0.05,0.15,0.09,0.06,0.06,0.04,0.71
10,Shot,129.05,191.48,-62.43,708,0.03,0.18,0.27,-0.09,0.05,0.08,-0.72
4,Gather,91.98,80.98,11.00,861,0.04,0.11,0.09,0.01,0.04,0.03,0.13
8,Knock On,74.28,59.65,14.64,221,0.01,0.34,0.27,0.07,0.03,0.02,0.17


In [62]:
team_data = get_team(exp_vaep_2023, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
9,Loose Ball Get,534.78,402.53,132.25,2158,0.12,0.25,0.19,0.06,0.25,0.19,-16.64
13,Uncontested Mark,443.10,444.65,-1.55,2140,0.12,0.21,0.21,-0.00,0.21,0.21,0.20
3,Free For,288.76,224.06,64.70,390,0.02,0.74,0.57,0.17,0.14,0.11,-8.14
1,Contested Mark,233.65,191.35,42.31,373,0.02,0.63,0.51,0.11,0.11,0.09,-5.32
0,Carry,230.88,284.73,-53.85,2723,0.15,0.08,0.10,-0.02,0.11,0.13,6.78
7,Kick,164.61,315.64,-151.03,4276,0.24,0.04,0.07,-0.04,0.08,0.15,19.01
6,Hard Ball Get,163.13,116.72,46.41,853,0.05,0.19,0.14,0.05,0.08,0.06,-5.84
10,Shot,92.89,143.36,-50.47,597,0.03,0.16,0.24,-0.08,0.04,0.07,6.35
11,Spoil,88.87,10.23,78.63,668,0.04,0.13,0.02,0.12,0.04,0.00,-9.90
4,Gather,88.49,84.29,4.20,665,0.04,0.13,0.13,0.01,0.04,0.04,-0.53


In [63]:
team_data = get_team(exp_vaep_2023, "Collingwood")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
9,Loose Ball Get,508.79,377.40,131.39,2210,0.11,0.23,0.17,0.06,0.23,0.16,-3.25
13,Uncontested Mark,442.29,438.52,3.77,2187,0.11,0.20,0.20,0.00,0.20,0.19,-0.09
0,Carry,351.96,412.71,-60.75,3287,0.16,0.11,0.13,-0.02,0.16,0.18,1.50
3,Free For,326.61,270.12,56.49,444,0.02,0.74,0.61,0.13,0.15,0.12,-1.40
1,Contested Mark,264.81,201.86,62.95,374,0.02,0.71,0.54,0.17,0.12,0.09,-1.56
7,Kick,166.18,334.47,-168.29,4530,0.23,0.04,0.07,-0.04,0.07,0.15,4.16
4,Gather,114.44,100.98,13.46,835,0.04,0.14,0.12,0.02,0.05,0.04,-0.33
6,Hard Ball Get,99.86,62.37,37.49,788,0.04,0.13,0.08,0.05,0.04,0.03,-0.93
10,Shot,86.81,143.32,-56.50,596,0.03,0.15,0.24,-0.09,0.04,0.06,1.40
8,Knock On,72.87,53.97,18.90,291,0.01,0.25,0.19,0.06,0.03,0.02,-0.47


In [64]:
team_data = get_team(exp_vaep_2023, "St Kilda")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
9,Loose Ball Get,508.36,348.55,159.81,2293,0.11,0.22,0.15,0.07,0.24,0.17,3.33
13,Uncontested Mark,470.83,457.81,13.02,2519,0.12,0.19,0.18,0.01,0.22,0.22,0.27
3,Free For,339.99,280.19,59.80,452,0.02,0.75,0.62,0.13,0.16,0.14,1.25
1,Contested Mark,246.19,182.53,63.66,378,0.02,0.65,0.48,0.17,0.12,0.09,1.33
0,Carry,235.41,294.78,-59.38,3415,0.16,0.07,0.09,-0.02,0.11,0.14,-1.24
6,Hard Ball Get,147.04,75.21,71.83,1015,0.05,0.14,0.07,0.07,0.07,0.04,1.50
7,Kick,119.42,275.35,-155.93,5027,0.24,0.02,0.05,-0.03,0.06,0.13,-3.25
10,Shot,114.80,158.27,-43.47,553,0.03,0.21,0.29,-0.08,0.05,0.08,-0.91
11,Spoil,66.64,-3.95,70.59,732,0.03,0.09,-0.01,0.10,0.03,-0.00,1.47
4,Gather,57.25,59.02,-1.77,794,0.04,0.07,0.07,-0.00,0.03,0.03,-0.04


Match Expectd Scores v Expected VAEP

In [65]:
match_id = "2022F4_Geelong_Sydney"
match_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id)
match_vaep_ranking.sort_values('exp_vaep_value', ascending = False).head()

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_exp_vaep_value,normalised_exp_offensive_value,normalised_exp_defensive_value,z_exp_vaep_value,z_exp_offensive_value,z_exp_defensive_value
30,Patrick Dangerfield,Geelong,15.338971,15.861482,-0.522511,58,1.0,1.0,26.0,1.000000,1.000000,0.456320,3.349271,3.697657,-0.105565
5,Chad Warner,Sydney,10.876419,11.711944,-0.835525,69,2.0,2.0,31.0,0.775568,0.768003,0.419641,2.146988,2.481509,-0.284229
24,Mark Blicavs,Geelong,9.543365,8.093971,1.449394,49,3.0,4.0,6.0,0.708525,0.565726,0.687389,1.787841,1.421153,1.019969
13,Isaac Smith,Geelong,9.473450,11.122511,-1.649062,69,4.0,3.0,38.0,0.705009,0.735049,0.324311,1.769005,2.308758,-0.748583
32,Robbie Fox,Sydney,7.458894,3.341723,4.117172,59,5.0,18.0,1.0,0.603693,0.300032,1.000000,1.226251,0.028364,2.542696


In [66]:
match_chain = get_match(exp_vaep_data, match_id)
match_chain.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value,Season
685607,2022F4_Geelong_Sydney,1,2.0,1,5.0,5.0,Geelong,Patrick Dangerfield,1.0,-2.0,1.0,-2.0,Gather,effective,NaN,0.982930,0.053575,0.000000,-0.000000,0.000000,2022
685608,2022F4_Geelong_Sydney,1,3.0,1,5.0,5.0,Geelong,Patrick Dangerfield,1.0,-2.0,17.0,-3.0,Kick,ineffective,NaN,0.904057,0.083576,-0.078873,-0.030002,-0.108875,2022
685609,2022F4_Geelong_Sydney,1,4.0,1,8.0,8.0,Geelong,Rhys Stanley,17.0,-3.0,18.0,-2.0,Loose Ball Get,effective,NaN,1.259172,0.061370,0.355115,0.022207,0.377322,2022
685610,2022F4_Geelong_Sydney,1,5.0,1,9.0,9.0,Geelong,Rhys Stanley,18.0,-2.0,-21.0,-3.0,Handball,clanger,NaN,0.000000,0.284538,-1.259172,-0.223169,-1.482340,2022
685611,2022F4_Geelong_Sydney,1,6.0,1,13.0,13.0,Sydney,Luke Parker,-21.0,-3.0,-25.0,-6.0,Hard Ball Get,effective,NaN,0.027917,0.262371,-0.256621,-0.262371,-0.518992,2022


In [67]:
match_chain.groupby('team').sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

,xScore,exp_scores,exp_concedes,exp_vaep_value,exp_offensive_value,exp_defensive_value
team,,,,,,
Geelong,105.861173,855.039689,103.883826,95.930948,96.717142,-0.786194
Sydney,42.689429,423.868839,161.646237,37.808585,52.550331,-14.741745


In [68]:
match_shots = match_chain[match_chain['xScore'] > 0]
match_shots.groupby('team').sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

,xScore,exp_scores,exp_concedes,exp_vaep_value,exp_offensive_value,exp_defensive_value
team,,,,,,
Geelong,105.861173,109.547698,6.680167,6.757945,10.638353,-3.880408
Sydney,42.689429,43.099144,2.811517,4.677702,6.474192,-1.796490


In [69]:
shots = exp_vaep_data[exp_vaep_data['xScore'] > 0]
shots.sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

xScore                 102339.145996
exp_scores             104175.254122
exp_concedes             5234.932323
exp_vaep_value           6906.887423
exp_offensive_value      9484.580526
exp_defensive_value     -2577.693101
dtype: object

In [70]:
exp_vaep_data[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']].sum()

xScore                 102339.145996
exp_scores             823959.057114
exp_concedes           167376.456941
exp_vaep_value         113298.669467
exp_offensive_value    111349.191252
exp_defensive_value      1949.478212
dtype: float64

In [71]:
exp_vaep_data.groupby('match_id').sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

,xScore,exp_scores,exp_concedes,exp_vaep_value,exp_offensive_value,exp_defensive_value
match_id,,,,,,
202101_BrisbaneLions_Sydney,162.196572,1292.239891,251.804485,181.117353,178.537805,2.579547
202101_Collingwood_WesternBulldogs,150.357535,1403.570922,269.355607,180.905160,170.775850,10.129309
202101_Essendon_Hawthorn,183.807470,1491.165443,303.755011,199.104978,196.646751,2.458227
202101_GreaterWesternSydney_StKilda,172.332941,1317.560496,308.437809,211.071873,197.982906,13.088968
202101_Melbourne_Fremantle,166.657732,1327.286007,281.260454,154.168718,163.897236,-9.728518
202101_NorthMelbourne_PortAdelaide,199.081580,1429.591257,277.696651,205.045092,209.576825,-4.531733
202101_Richmond_Carlton,206.995237,1477.458889,270.228288,216.671428,219.042615,-2.371188
202101_WestCoast_GoldCoast,164.258414,1238.533427,248.332142,193.765406,184.843320,8.922086
202102_Carlton_Collingwood,166.299330,1393.586679,267.437072,215.801651,195.371327,20.430323
